In [ ]:
# import library ที่จำเป็น
import pandas as pd
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader

In [ ]:
# นิยามตัวแปร device สำหรับการใช้งานใน gpu หาก available ถ้าไม่มีใช้ cpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

True


In [ ]:
# นิยามสำหรับ categorical value เป็น index

REPLACE_CATS = {
    "metal_linker":[1, 2, 3, 4, 9, 10, 12], 
    'organic_linker1':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 
    'organic_linker2':[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 38, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59], 
    'functional_groups':['Br-NHMe', 'I-OH', 'Br-OPr', 'Et-H', 'OPr', 'NO2-Me', 'SO3H-NH2', 'NH2-F', 'NO2-Pr', 'Br-Pr', 'Ph-Et', 'OH', 'OEt-Et', 'Pr-Et', 'OH-SO3H', 'Et-Me', 'OH-Et', 'I-F', 'Pr-COOH', 'SO3H-F', 'Ph-OH', 'OH-OMe', 'H-Pr', 'COOH-Me', 'COOH-F', 'Et-SO3H', 'COOH-OMe', 'Me-NHMe', 'F-CN', 'NO2-Ph', 'Pr-F', 'CN-NO2', 'H-OH', 'Me-HCO', 'Ph-Cl', 'COOH-OPr', 'OH-HCO', 'Pr-OEt', 'OPr-Et', 'OH-Pr', 'Cl-OH', 'OMe-COOH', 'OMe-H', 'SO3H-NHMe', 'H-NO2', 'OPr-OEt', ' ', 'NO2-NH2', 'OPr-CN', 'I-Ph', 'H-Et', 'NH2-OMe', 'OH-H', 'Me-SO3H', 'OEt-H', 'Pr-I', 'F-Et', 'COOH-NHMe', 'HCO-Me', 'CN-NHMe', 'NO2-CN', 'Pr-OPr', 'CN-HCO', 'Br-COOH', 'OPr-Br', 'OH-Ph', 'OEt-SO3H', 'Cl-SO3H', 'NO2-HCO', 'Cl-CN', 'Pr-Me', 'NH2-NHMe', 'Pr-HCO', 'OEt-NHMe', 'NHMe-Pr', 'I-HCO', 'OEt-CN', 'Ph-COOH', 'Cl-Me', 'Et-NH2', 'Et', 'Ph-SO3H', 'F', 'Ph-CN', 'OEt-OMe', 'F-NH2', 'F-SO3H', 'NO2-I', 'HCO-Br', 'OEt-HCO', 'NHMe-Me', 'HCO-CN', 'Br-H', 'CN', 'Cl-OEt', 'HCO-COOH', 'Et-I', 'H-COOH', 'F-OEt', 'Pr-SO3H', 'COOH-CN', 'Et-OH', 'H-NHMe', 'SO3H-OH', 'Pr-Cl', 'OEt-OPr', 'Me-Et', 'Ph-H', 'SO3H-CN', 'Ph-NH2', 'Br-Me', 'Et-OPr', 'CN-H', 'NH2-HCO', 'Et-OMe', 'I-Br', 'Ph-OPr', 'H-Ph', 'NHMe-NO2', 'Et-Ph', 'I-CN', 'COOH-H', 'CN-I', 'COOH-OH', 'Et-Br', 'I-NO2', 'OEt-NH2', 'SO3H-Cl', 'COOH-NO2', 'SO3H-OEt', 'Me', 'OPr-COOH', 'H-SO3H', 'OPr-Cl', 'SO3H-Ph', 'CN-OH', 'Cl-Ph', 'Br-OMe', 'Me-CN', 'OH-Me', 'SO3H-Me', 'Me-OH', 'CN-COOH', 'F-NO2', 'Ph-OMe', 'NHMe-COOH', 'CN-OEt', 'OEt-Me', 'OMe-Cl', 'OMe-NO2', 'F-OPr', 'NHMe-Cl', 'NH2-H', 'H-CN', 'OMe-SO3H', 'Ph-F', 'F-H', 'F-Cl', 'OEt-Br', 'F-Br', 'SO3H-NO2', 'HCO-NH2', 'HCO-NHMe', 'SO3H', 'NO2-NHMe', 'NH2-NO2', 'NO2-OPr', 'NH2-Ph', 'H-Cl', 'Ph', 'H-NH2', 'Br', 'Ph-OEt', 'Cl-Pr', 'I-Et', 'OH-Br', 'NO2-SO3H', 'NHMe-HCO', 'Cl-I', 'Pr-OMe', 'NH2-OEt', 'OPr-Me', 'NO2-Et', 'OMe-Pr', 'Me-NH2', 'Br-OH', 'H-OMe', 'Me-OEt', 'SO3H-Et', 'Et-Pr', 'OPr-NO2', 'OMe-I', 'Cl-Br', 'NO2-OMe', 'CN-NH2', 'Ph-NO2', 'Pr-Br', 'NO2', 'Et-COOH', 'Pr-Ph', 'HCO-H', 'OPr-H', 'Ph-Me', 'HCO-NO2', 'NHMe-Br', 'NHMe-NH2', 'OPr-SO3H', 'Br-NO2', 'I-Pr', 'NHMe-H', 'F-Ph', 'Cl-OPr', 'HCO-OMe', 'COOH-Br', 'OEt-OH', 'OPr-I', 'NH2-Me', 'Me-COOH', 'H-OPr', 'SO3H-OMe', 'OMe-OH', 'Me-NO2', 'HCO-OPr', 'NHMe-OPr', 'OH-OPr', 'COOH-NH2', 'Ph-NHMe', 'HCO', 'NHMe', 'SO3H-Br', 'CN-Et', 'F-OH', 'I', 'HCO-Ph', 'NHMe-SO3H', 'NO2-OEt', 'OH-NHMe', 'COOH-HCO', 'Cl-NH2', 'NHMe-CN', 'F-NHMe', 'I-SO3H', 'Me-OPr', 'SO3H-I', 'Cl-OMe', 'NH2-Pr', 'Pr-OH', 'COOH-OEt', 'NH2', 'H-I', 'Cl', 'NO2-F', 'Cl-F', 'NH2-Cl', 'Br-OEt', 'Pr', 'Ph-I', 'OH-OEt', 'OMe-CN', 'OMe-NHMe', 'Me-I', 'NH2-SO3H', 'OMe-F', 'NHMe-I', 'COOH', 'OH-COOH', 'SO3H-HCO', 'Et-HCO', 'NH2-COOH', 'Br-CN', 'I-Cl', 'CN-Ph', 'I-NHMe', 'Br-SO3H', 'NO2-H', 'OMe-Ph', 'H-HCO', 'NH2-OH', 'Cl-HCO', 'OMe-Me', 'I-OMe', 'CN-OPr', 'Pr-CN', 'F-OMe', 'HCO-OH', 'HCO-Cl', 'Cl-COOH', 'OEt-I', 'Br-Et', 'OMe-OPr', 'F-Pr', 'Me-F', 'COOH-SO3H', 'NHMe-Ph', 'NHMe-OH', 'CN-Me', 'Pr-H', 'HCO-SO3H', 'Me-Br', 'F-COOH', 'I-OPr', 'OPr-HCO', 'NHMe-Et', 'I-COOH', 'HCO-I', 'OPr-OMe', 'OMe-Br', 'NH2-CN', 'OMe-NH2', 'CN-F', 'OEt-F', 'F-HCO', 'H-OEt', 'OH-NO2', 'Cl-NHMe', 'OMe-HCO', 'OPr-Ph', 'OPr-F', 'NH2-OPr', 'OEt-Ph', 'HCO-Pr', 'Br-HCO', 'Cl-Et', 'NO2-COOH', 'SO3H-COOH', 'H-Me', 'OMe-Et', 'CN-OMe', 'NO2-OH', 'COOH-Pr', 'Pr-NH2', 'OH-NH2', 'OPr-NHMe', 'Cl-H', 'F-I', 'Br-NH2', 'Et-F', 'OEt-COOH', 'CN-SO3H', 'HCO-Et', 'CN-Br', 'NH2-Br', 'I-H', 'COOH-I', 'Ph-Br', 'OH-CN', 'Et-NO2', 'OMe-OEt', 'I-NH2', 'CN-Pr', 'NHMe-OEt', 'H-Br', 'OPr-OH', 'OH-Cl', 'Et-NHMe', 'OEt-Cl', 'Br-Ph', 'HCO-F', 'Br-Cl', 'COOH-Cl', 'Pr-NO2', 'I-Me', 'Me-OMe', 'F-Me', 'NHMe-OMe', 'Br-F', 'SO3H-OPr', 'OEt-NO2', 'Et-Cl', 'NH2-Et', 'OH-F', 'NHMe-F', 'OPr-NH2', 'SO3H-Pr', 'HCO-OEt', 'Me-Pr', 'OEt-Pr', 'OH-I', 'Pr-NHMe', 'Ph-HCO', 'I-OEt', 'COOH-Ph', 'OEt', 'Cl-NO2', 'Me-Cl', 'OMe', 'Et-CN', 'OPr-Pr', 'NO2-Br', 'Ph-Pr', 'NO2-Cl', 'COOH-Et', 'CN-Cl', 'Me-H', 'Me-Ph', 'H-F', 'Et-OEt', 'NH2-I', 'SO3H-H', 'H', 'Br-I'],
    'functional_groups-1':['NH2', 'HCO', 'Et', 'NO2', 'NHMe', 'Cl', 'OPr', 'F', 'I', 'OEt', 'OH', 'Pr', 'OMe', ' ', 'Me', 'COOH', 'CN', 'SO3H', 'Ph', 'Br', 'H'], 
    'functional_groups-2':['NH2', 'HCO', 'Et', 'NO2', 'NHMe', 'Cl', 'OPr', 'F', 'I', 'OEt', 'OH', 'Pr', 'OMe', ' ', 'Me', 'COOH', 'CN', 'SO3H', 'Ph', 'Br', 'H'], 
    "topology":['acs', 'bcu', 'etb', 'fof', 'nbo', 'pcu', 'pts', 'rht', 'sra', 'tbo', 'the'],
}

# นิยามฟังก์ชั่นสำหรับแปลงค่า categorical value เป็น index
def replace_cats(type, df):
  return df.replace(REPLACE_CATS[type], list(range(len(REPLACE_CATS[type])))).astype('int64')

# นิยามฟังก์ชั่นสำหรับ return ความยาวของ categorical values ที่มีทั้งหมดในคอลัมน์
def len_cats(type):
  return len(REPLACE_CATS[type])

# นิยามฟังก์ชั่นสำหรับการหารค่า metal linker, organic linker, functional groups ด้วย unit cell volume
def update_data(data):
    for e in ['n_ml', 'n_ml_m_n', '_n_ol1', '_n_ol2', '_n_fg1', '_n_fg2']:
        data[e] = data[e] / data['volume [A^3]']
    return data


In [ ]:
# คอลัมน์สำหรับ catagorical values
cat_cols = [ 'topology', 'functional_groups-1', 'functional_groups-2', 'metal_linker', 'organic_linker1', 'organic_linker2']

# คอลัมน์สำหรับ continuous values
cont_cols = [    
    'NUM_ATOMS', # จำนวนอะตอมรวม
    'Br', 'F', 'Cr', 'Cu', 'N', 'H', 'Zn', 'O', 'P', 'V', 'S', 'Ba', 'C', 'I', 'Ni', 'Cl', # จำนวนอะตอมธาตุ / จำนวนอะตอมรวม
    'sc', 
    'n_ml', # สัดส่วน metal linker / unit cell volume
    'n_m', 'n_ml_m_n', 
    
    # เป็นค่า Radial Distribution RDF-R-i โดย R คือ Radius และ i = 1 คือ electronegativity, 2 คือ polarization, 3 คือ van_der_waals volume
    'RDF-2-1', 'RDF-2-2', 'RDF-2-3', 
    'RDF-2.2-1', 'RDF-2.2-2', 'RDF-2.2-3', 
    'RDF-2.5-1', 'RDF-2.5-2', 'RDF-2.5-3', 
    'RDF-2.8-1', 'RDF-2.8-2', 'RDF-2.8-3',              
    'RDF-3-1', 'RDF-3-2', 'RDF-3-3', 
    'RDF-3.2-1', 'RDF-3.2-2', 'RDF-3.2-3',              
    'RDF-3.5-1', 'RDF-3.5-2', 'RDF-3.5-3', 
    'RDF-3.8-1', 'RDF-3.8-2', 'RDF-3.8-3',             
    'RDF-4-1', 'RDF-4-2', 'RDF-4-3',
    'RDF-4.5-1', 'RDF-4.5-2', 'RDF-4.5-3',
    'RDF-5-1', 'RDF-5-2', 'RDF-5-3', 
    'RDF-5.5-1', 'RDF-5.5-2', 'RDF-5.5-3',
    'RDF-6-1', 'RDF-6-2', 'RDF-6-3',
    'RDF-6.5-1', 'RDF-6.5-2', 'RDF-6.5-3', 
    'RDF-7-1', 'RDF-7-2', 'RDF-7-3',
    'RDF-7.5-1', 'RDF-7.5-2', 'RDF-7.5-3',
    'RDF-8-1', 'RDF-8-2', 'RDF-8-3',
    'RDF-8.5-1', 'RDF-8.5-2', 'RDF-8.5-3',
    'RDF-9-1', 'RDF-9-2', 'RDF-9-3', 
    'RDF-9.5-1', 'RDF-9.5-2', 'RDF-9.5-3', 
    'RDF-10-1', 'RDF-10-2', 'RDF-10-3',
    'RDF-10.5-1', 'RDF-10.5-2', 'RDF-10.5-3',
    'RDF-11-1', 'RDF-11-2', 'RDF-11-3',   
    'RDF-11.5-1', 'RDF-11.5-2', 'RDF-11.5-3',             
    'RDF-12-1', 'RDF-12-2', 'RDF-12-3', 
    'RDF-12.5-1', 'RDF-12.5-2', 'RDF-12.5-3',
    'RDF-13-1', 'RDF-13-2', 'RDF-13-3',
    'RDF-13.5-1', 'RDF-13.5-2', 'RDF-13.5-3',
    'RDF-14-1', 'RDF-14-2', 'RDF-14-3', 
    'RDF-14.5-1', 'RDF-14.5-2', 'RDF-14.5-3',
    'RDF-15-1', 'RDF-15-2', 'RDF-15-3', 
    'RDF-15.5-1', 'RDF-15.5-2', 'RDF-15.5-3',
    'RDF-16-1', 'RDF-16-2', 'RDF-16-3', 
    'RDF-16.5-1', 'RDF-16.5-2', 'RDF-16.5-3',
    'RDF-17-1', 'RDF-17-2', 'RDF-17-3',
    'RDF-17.5-1', 'RDF-17.5-2', 'RDF-17.5-3',
    'RDF-18-1', 'RDF-18-2', 'RDF-18-3', 
    'RDF-18.5-1', 'RDF-18.5-2', 'RDF-18.5-3',
    'RDF-19-1', 'RDF-19-2', 'RDF-19-3',
    'RDF-19.5-1', 'RDF-19.5-2', 'RDF-19.5-3',
    'RDF-20-1', 'RDF-20-2', 'RDF-20-3', 
    'RDF-20.5-1', 'RDF-20.5-2', 'RDF-20.5-3',
    'RDF-21-1', 'RDF-21-2', 'RDF-21-3',
    'RDF-21.5-1', 'RDF-21.5-2', 'RDF-21.5-3',
    'RDF-22-1', 'RDF-22-2', 'RDF-22-3', 
    'RDF-22.5-1', 'RDF-22.5-2', 'RDF-22.5-3',
    'RDF-23-1', 'RDF-23-2', 'RDF-23-3',
    'RDF-23.5-1', 'RDF-23.5-2', 'RDF-23.5-3',
    'RDF-24-1', 'RDF-24-2', 'RDF-24-3', 
    'RDF-24.5-1', 'RDF-24.5-2', 'RDF-24.5-3',
    'RDF-25-1', 'RDF-25-2', 'RDF-25-3', 
    'RDF-25.5-1', 'RDF-25.5-2', 'RDF-25.5-3',
    'RDF-26-1', 'RDF-26-2', 'RDF-26-3', 
    'RDF-26.5-1', 'RDF-26.5-2', 'RDF-26.5-3',
    'RDF-27-1', 'RDF-27-2', 'RDF-27-3',      
    'RDF-27.5-1', 'RDF-27.5-2', 'RDF-27.5-3',        
    'RDF-28-1', 'RDF-28-2', 'RDF-28-3', 
    'RDF-28.5-1', 'RDF-28.5-2', 'RDF-28.5-3',
    'RDF-29-1', 'RDF-29-2', 'RDF-29-3',      
    'RDF-29.5-1', 'RDF-29.5-2', 'RDF-29.5-3',                     
    'RDF-30-1', 'RDF-30-2', 'RDF-30-3', 
    
        'DEG_UNSAT', 'DEG_UNSAT_C', 'METAL_C', 'O_METAL', 'ELEC_FRAC', 'W_ELEC', 'N_C', 'O_C', 'N_O',
    'abc', # ค่าเฉลี่ยของ unit cell lengths
    'alpha-beta-gamma', # ค่าเฉลี่ยของ unit cell angles
    # ส่วนถัดไปเป็นฟีเจอร์จาก dataset โดย คำนวณหา density [g/cm^3] 
    'void_fraction', 'CO2/N2_selectivity', 'heat_adsorption_CO2_P0.15bar_T298K [kcal/mol]', 'density [g/cm^3]', 
    'volume [A^3]', 'weight [u]', 'surface_area [m^2/g]', 'void_volume [cm^3/g]', 'density [g/cm^3]'
]

In [ ]:
# นิยาม Model
class MyModel(nn.Module):

    def __init__(self, emb_szs, n_cont, out_sz, layers, p=0.5):
        super().__init__()
    
        # สร้าง embedding layers โดยอิงจาก input size, output size ที่คำนวณไว้
        self.embeds = nn.ModuleList([nn.Embedding(ni, nf) for ni,nf in emb_szs])
        
        # นิยาม drop out
        self.emb_drop = nn.Dropout(p)
        
        # นิยาม batchnorm1d สำหรับ continous values        
        self.bn_cont = nn.BatchNorm1d(n_cont)
        
        # นิยาม output size จาก embedding layer
        n_emb = sum((nf for ni,nf in emb_szs))
        
        # นิยาม input size ก่อนเข้า MLPs
        n_in1 = n_emb + n_cont

        layerlist = []        
        for i in layers:
            layerlist.append(nn.Linear(n_in1, i)) 
            
            #ใช้ LeakyReLU เป็น activation function โดยมีค่าความชันด้านลบเป็น 0.2
            layerlist.append(nn.LeakyReLU(0.2, inplace=True))

            #ใส่ BatchNorm1d ให้ output            
            layerlist.append(nn.BatchNorm1d(i))
            
            # ใส่ drop out
            layerlist.append(nn.Dropout(p))
            
            n_in1 = i
        
        # กำหนด output layer
        layerlist.append(nn.Linear(layers[-1],out_sz))
        
        # นำ layer ที่สร้างไว้มาทำเป็น Sequential    
        self.layers = nn.Sequential(*layerlist)
    
    def forward(self, x_cat, x_cont):
        
        # นำ categorical values แต่ละคอลัมน์ มาผ่าน embedding layer แล้วเก็บไว้ในตัวแปร embeddings
        embeddings = []
        for i, e in enumerate(self.embeds):
            embeddings.append(e(x_cat[:,i]))
            
        # นำ categorical values ที่ผ่าน embedding layer มารวมกัน
        x = torch.cat(embeddings, 1)
        
        # นำไปผ่าน dropout
        x = self.emb_drop(x)
        
        # นำ continous values ไปผ่าน batchnorm1d
        x_cont = self.bn_cont(x_cont)

        # นำ categorical values และ continous values มาต่อกัน
        x = torch.cat([x, x_cont], 1)

        # นำ input ไปเข้า MLPs
        x = self.layers(x)

        # คืนค่า output
        return x

In [ ]:
from sklearn.model_selection import train_test_split

# ใช้หลักการ cross validation

MAX_MODELS = 10

mse_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

initial_lr = 0.01
epochs = 10000
patient = 200

dataset = pd.concat([pd.read_csv('./trainset.csv'), pd.read_csv('./testset.csv')])

MODEL = {}

mse_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

for i in range(MAX_MODELS):
    trainset, testset = train_test_split(dataset.copy(), test_size=0.2)

    trainset = update_data(trainset)
    testset = update_data(testset)

    for col in cat_cols:
        trainset[col] = replace_cats(col, trainset[col])
        testset[col] = replace_cats(col, testset[col])

    X_train_cats = torch.tensor(np.stack([trainset[col].values for col in cat_cols], 1), dtype=torch.int64).to(device)
    X_train_conts = torch.tensor(np.stack([trainset[col].values for col in cont_cols], 1), dtype=torch.float).to(device)

    X_test_cats = torch.tensor(np.stack([testset[col].values for col in cat_cols], 1), dtype=torch.int64).to(device)
    X_test_conts = torch.tensor(np.stack([testset[col].values for col in cont_cols], 1), dtype=torch.float).to(device)

    y_train = torch.tensor(trainset['CO2_working_capacity [mL/g]'].values, dtype=torch.float).reshape(-1,1).to(device)
    y_test = torch.tensor(testset['CO2_working_capacity [mL/g]'].values, dtype=torch.float).reshape(-1,1).to(device)

    cat_szs = [len_cats(col) for col in cat_cols]
    emb_szs = [(size, min(50, (size+1)//2)) for size in cat_szs]

    layers = [4096, 2048, 1024, 512, 1024, 2048, 4096, 1024]
    model = MyModel(emb_szs, X_train_conts.shape[1], 1, layers, p=0.55).to(device)

    MODEL[i] = {
      'stop':False,
      'X_train_cats':X_train_cats, 
      'X_train_conts':X_train_conts,
      'X_test_cats':X_test_cats,
      'X_test_conts':X_test_conts,
      'y_train':y_train, 
      'y_test':y_test, 
      'train_loader':DataLoader(list(zip(X_train_cats, X_train_conts, y_train)), batch_size=1024, shuffle=True),
      'model':model,
      'optimizer':torch.optim.Adam(model.parameters(), initial_lr),
      'model_mape_min':[{'mape':1e6, 'lmae':1e6, 'err':1e6, 'model':None} for i in range(10)],
      'model_lmae_min':[{'mape':1e6, 'lmae':1e6, 'err':1e6, 'model':None} for i in range(10)],
      'val_mape_min':1e6, 
      'val_lmae_min':1e6, 
      'err_mape':0, 
      'err_lmae':0,
      'epoch_min':0,
      'train_loss':0,
      'val_loss':0
    }

In [ ]:
# รายละเอียดการทำงานหลักๆ ดูได้จาก tlmcc_model.ipynb ไฟล์นี้แค่ทำการ train พร้อมกับหลายโมเดลด้วย dataset ที่ต่างกัน
for i in range(epochs):
    i+=1

    for j, m in enumerate(MODEL):

        if MODEL[m]['stop'] == False:

            MODEL[m]['model'].train()
            batch_loss = []
            for x_cats, x_conts, y in MODEL[m]['train_loader']:
                y_pred = MODEL[m]['model'](x_cats, x_conts)

                MAE = l1_loss(y_pred, y)
                MAPE = 2*(torch.abs(y_pred - y)/(torch.abs(y_pred)+torch.abs(y))).mean()
                train_loss = torch.log(MAE + 5*MAPE)

                MODEL[m]['optimizer'].zero_grad()
                train_loss.backward()
                MODEL[m]['optimizer'].step()

                batch_loss.append(torch.log10(MAE).item())

            MODEL[m]['train_loss'] = np.mean(batch_loss)

            MODEL[m]['model'].eval()

            with torch.no_grad():
                y_val = MODEL[m]['model'](MODEL[m]['X_test_cats'], MODEL[m]['X_test_conts'])      

                val_lmae = torch.log10(l1_loss(y_val, MODEL[m]['y_test']))
                val_mape = 2*(torch.abs(y_val - MODEL[m]['y_test'])/(torch.abs(y_val)+torch.abs(MODEL[m]['y_test']))).mean()

                MODEL[m]['val_loss'] = val_lmae.item()

                if MODEL[m]['val_loss'] < MODEL[m]['val_lmae_min']:
                    MODEL[m]['err_lmae'] = 100*torch.abs((y_val-MODEL[m]['y_test'])/MODEL[m]['y_test']).mean().item()
                    max_model = ([m1['lmae'] for m1 in MODEL[m]['model_lmae_min']])
                    MODEL[m]['model_lmae_min'][np.argmax(max_model)] = {'mape':val_mape, 'lmae':MODEL[m]['val_loss'], 'err':MODEL[m]['err_lmae'] , 'model':model.state_dict()}
                    max_model = ([m1['lmae'] for m1 in MODEL[m]['model_lmae_min']])
                    MODEL[m]['val_lmae_min'] = np.max(max_model)
                    MODEL[m]['epoch_min'] = i
                elif val_mape < MODEL[m]['val_mape_min']:
                    MODEL[m]['err_mape'] = 100*torch.abs((y_val-MODEL[m]['y_test'])/MODEL[m]['y_test']).mean().item() 
                    max_model = ([m1['mape'] for m1 in MODEL[m]['model_mape_min']])
                    MODEL[m]['model_mape_min'][np.argmax(max_model)] = {'mape':val_mape, 'lmae':MODEL[m]['val_loss'], 'err':MODEL[m]['err_mape'], 'model':model.state_dict()}
                    max_model = ([m1['mape'] for m1 in MODEL[m]['model_mape_min']])                    
                    MODEL[m]['val_mape_min'] = np.max(max_model)
                    MODEL[m]['epoch_min'] = i

            if i - MODEL[m]['epoch_min'] > patient:
                MODEL[m]['stop'] = True

            lr = initial_lr * (0.95 ** (i // 10))

            for param_group in MODEL[m]['optimizer'].param_groups:
                param_group['lr'] = lr
    
            if i%10 == 0:
                print(f"epoch: {i:3}  model: {j:3}  train_loss: {MODEL[m]['train_loss']:10.8f} val_loss: {MODEL[m]['val_loss']:10.8f}  min_lmae: {MODEL[m]['val_lmae_min']:10.8f} err_min_lmae: {MODEL[m]['err_lmae']:10.8f}%  status:{str(not MODEL[m]['stop'])}")

    if not (False in [MODEL[m]['stop'] for m in MODEL]):
        print("DONE")
        break

print(f"epoch: {i:3}  model: {i:3}  train_loss: {MODEL[m]['train_loss']:10.8f} val_loss: {MODEL[m]['val_loss']:10.8f}  min_lmae: {MODEL[m]['val_lmae_min']:10.8f} err_min_lmae: {MODEL[m]['err_lmae']:10.8f}%")

In [ ]:
data_test = pd.read_csv('test_fixed.csv')

data_test = update(data_test)

for col in cat_cols:
    data_test[col] = replace_cats(col, data_test[col])

data_test_cats = torch.tensor(np.stack([data_test[col].values for col in cat_cols], 1), dtype=torch.int64).to(device)
data_test_conts = torch.tensor(np.stack([data_test[col].values for col in cont_cols], 1), dtype=torch.float).to(device)

pred = torch.zeros((data_test_cats.shape[0], 1)).to(device)
count = 0

for j, m in enumerate(MODEL):
    for mname, model_list in [('LMAE', MODEL[m]['model_lmae_min'])]:
        for i, mlist in enumerate(model_list):    
            if mlist['model'] == None:
                break
        print(f"\t{i}: val_lmae = {mlist['lmae']:10.8f} | val_mpae = {mlist['mape']:10.8f} | err = {mlist['err']:10.8f}")
        MODEL[m]['model'].load_state_dict(mlist['model'])
        MODEL[m]['model'].eval()
        with torch.no_grad():
            pred += MODEL[m]['model'](data_test_cats, data_test_conts)
            count+=1

pred /= count


submission = pd.DataFrame({
    "id": [str(68614+i) for i in range(17000)],
    "CO2_working_capacity [mL/g]": pred.cpu().numpy().T[0]
})

submission.to_csv('./submission.csv', index=False)
